In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.ensemble import RandomForestClassifier

In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [4]:
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

/resources/common/.virtualenv/python3/lib/python3.4/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
trainn = pd.read_csv('train.csv')
testt = pd.read_csv('test.csv')

In [6]:
trainn.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,Rear,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,Rear,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,Front,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,Rear,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,Rear,208,0.93,54,58,Rear


In [7]:
testt.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,2c9180975a056a64015a1e10d3f270fe,Right,67,0.63,107,169
1,2c9180975a056a64015a1de4deb16bdc,Front,16,0.88,61,69
2,2c9180975a056a64015a1e0e70ea70ce,Right,44,1.15,232,202
3,2c9180975a056a64015a1dfed0c46ec6,Right,50,1.10,137,125
4,2c9180975a056a64015a1dfed0c46ec7,Front,30,0.95,99,104


In [8]:
trainn.info()
print('_'*40)
testt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38485 entries, 0 to 38484
Data columns (total 7 columns):
Id                     38485 non-null object
DetectedCamera         38485 non-null object
AngleOfSign            38485 non-null int64
SignAspectRatio        38485 non-null float64
SignWidth              38485 non-null int64
SignHeight             38485 non-null int64
SignFacing (Target)    38485 non-null object
dtypes: float64(1), int64(3), object(3)
memory usage: 2.1+ MB
________________________________________
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31485 entries, 0 to 31484
Data columns (total 6 columns):
Id                 31485 non-null object
DetectedCamera     31485 non-null object
AngleOfSign        31485 non-null int64
SignAspectRatio    31485 non-null float64
SignWidth          31485 non-null int64
SignHeight         31485 non-null int64
dtypes: float64(1), int64(3), object(2)
memory usage: 1.4+ MB


In [9]:
def num_missing(x):
    return sum(x.isnull())
"Missing values per column for test data set"
testt.apply(num_missing, axis=0)

Id                 0
DetectedCamera     0
AngleOfSign        0
SignAspectRatio    0
SignWidth          0
SignHeight         0
dtype: int64

In [10]:
def num_missing(x):
    return sum(x.isnull())
"Missing values per column for train data set"
trainn.apply(num_missing, axis=0)

Id                     0
DetectedCamera         0
AngleOfSign            0
SignAspectRatio        0
SignWidth              0
SignHeight             0
SignFacing (Target)    0
dtype: int64

In [11]:
trainn.describe()

,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
count,38485.000000,38485.000000,38485.000000,38485.000000
mean,132.930986,0.901241,92.922957,104.993140
std,98.042472,0.226068,52.399274,53.347424
min,1.000000,0.260000,19.000000,23.000000
25%,42.000000,0.770000,54.000000,65.000000
50%,80.000000,0.930000,80.000000,96.000000
75%,228.000000,1.040000,118.000000,132.000000
max,360.000000,2.790000,589.000000,513.000000


In [12]:
trainn['DetectedCamera'].value_counts()

Front    10910
Right    10516
Left      9298
Rear      7761
Name: DetectedCamera, dtype: int64

In [13]:
mapping = {'Front':0,'Right':1,'Left':2,'Rear':3}
trainn = trainn.replace({'DetectedCamera':mapping})
testt = testt.replace({'DetectedCamera':mapping})

In [14]:
trainn.rename(columns = {'SignFacing (Target)': 'Target'}, inplace=True)

In [15]:
mapping = {'Front':0, 'Left':1, 'Rear':2, 'Right':3}
trainn = trainn.replace({'Target':mapping})

In [16]:
y_train = trainn['Target']
test_id = testt['Id']

In [17]:
trainn.drop(['Target','Id'], inplace=True, axis=1)
testt.drop('Id',inplace=True,axis=1)

In [18]:
# Random Forest

random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(trainn, y_train)
Y_pred = random_forest.predict(testt)
random_forest.score(trainn, y_train)
acc_random_forest = round(random_forest.score(trainn, y_train) * 100, 2)
acc_random_forest

99.920000000000002

In [19]:
#XGBoost
model_xg = XGBClassifier()
model_xg.fit(trainn, y_train)
Y_pred = model_xg.predict(testt)
acc_xg = round(model_xg.score(trainn, y_train) * 100, 2)
acc_xg

96.359999999999999

In [20]:
pred = model_xg.predict_proba(testt)

In [49]:
columns = ['Front','Left','Rear','Right']
sub = pd.DataFrame(data=pred, columns=columns)
sub['Id'] = test_id
sub = sub[['Id','Front','Left','Rear','Right']]
sub.to_csv("/resources/data/Hackerearth/Sign_predict/sub_xg.csv", index=False)